In [1]:
#dependencies and setup
import pandas as pd
import os
from openpyxl import Workbook
import numpy as np
import requests
import json
import re
from pprint import pprint
from bs4 import BeautifulSoup
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import time
# SQLite dependencies
import sqlite3
from sqlalchemy import create_engine
from sqlalchemy import Column, Integer, String, Float

In [2]:
# create a browser instance using splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)
time.sleep(1)



====== WebDriver manager ======
Current google-chrome version is 110.0.5481
Get LATEST chromedriver version for 110.0.5481 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/110.0.5481.77/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\mosab\.wdm\drivers\chromedriver\win32\110.0.5481.77]


# Summary

In [37]:
# visit the NJ schools url
nj_school_rate_URL = 'https://www.schooldigger.com/go/NJ/schoolrank.aspx'
browser.visit(nj_school_rate_URL)
time.sleep(5)
# use splinter to click on the button to go to next page
browser.find_by_xpath('//*[@id="divWidgetParams"]/div/div[2]/button').click()
time.sleep(1)
browser.find_by_xpath('//*[@id="divWidgetParams"]/div/div[2]/ul/li[2]/a').click()
time.sleep(4)
browser.find_by_xpath('//*[@id="tabSchooList_length"]/label/select').click()
time.sleep(3)
browser.find_by_xpath('//*[@id="tabSchooList_length"]/label/select/option[4]').click()
time.sleep(1)

In [229]:
# create HTML object
html = browser.html
# parse HTML with BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')
total_records = int(soup.find('span' ,class_='tRows').text)
total_records

1319

In [230]:
total_pages = int(np.ceil(total_records/100))
total_pages

14

In [231]:
rank_dfs=[]
for x in range(1,total_pages+1):
    # create HTML object
    html = browser.html
    # parse HTML with BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')
    # access html elements to get the link for the table data
    div = soup.find(id= 'tabSchooList_wrapper')
    table = div.find('table', id = 'tabSchooList')
    tbody = table.find('tbody')
    listings= tbody.find_all('td')
    rank = []
    school = []
    grades = []
    district = []
    students = []
    free_lunch_rec=[]
    total_exp=[]
    score=[]
    year = []
    for i in range(0,len(listings),10):
        rank.append(listings[i].text) 
        year.append(2017)
    for i in range(1,len(listings),10):
        school.append(listings[i].text.replace('\n',''))     
    for i in range(2,len(listings),10):
        grades.append(listings[i].text.replace('\n',''))
    for i in range(3,len(listings),10):
        district.append(listings[i].text.replace('\n',''))
    for i in range(4,len(listings),10):
        students.append(listings[i].text.replace('\n',''))
    for i in range(5,len(listings),10):
        free_lunch_rec.append(listings[i].text.replace('\n',''))
    for i in range(6,len(listings),10):
        total_exp.append(listings[i].text.replace('\n',''))
    for i in range(7,len(listings),10):
        score.append(listings[i].text.replace('\n',''))

    df = pd.DataFrame({'rank':rank, 'school':school, 'grades':grades, 'district':district, 'students':students,\
                       'free_lunch_rec':free_lunch_rec, 'total_exp':total_exp, 'score':score, 'year':year})
    rank_dfs.append(df)
    print(f'Page {x} scraped!')
    # use splinter to click on the button to go to next page
    if x == total_pages:
        break
    elif x <= 4:
        browser.find_by_xpath(f'//*[@id="tabSchooList_paginate"]/ul/li[{x+1}]/a').click()
    elif (x > 4 and x < int(np.ceil(total_records/100))-2):
        browser.find_by_xpath('//*[@id="tabSchooList_paginate"]/ul/li[5]/a').click()
    elif (x > 4 and x == int(np.ceil(total_records/100))-2) or (x==5 and total_pages <=7):
        browser.find_by_xpath('//*[@id="tabSchooList_paginate"]/ul/li[6]/a').click()
    elif (x > 4 and x == int(np.ceil(total_records/100))-1) or (x==6 and total_pages <=7):
        browser.find_by_xpath('//*[@id="tabSchooList_paginate"]/ul/li[7]/a').click()    
    else:
        continue
    time.sleep(5)
rank_df = pd.concat(rank_dfs,ignore_index = True)
rank_df.head()

Page 1 scraped!
Page 2 scraped!
Page 3 scraped!
Page 4 scraped!
Page 5 scraped!
Page 6 scraped!
Page 7 scraped!
Page 8 scraped!
Page 9 scraped!
Page 10 scraped!
Page 11 scraped!
Page 12 scraped!
Page 13 scraped!
Page 14 scraped!


,rank,school,grades,district,students,free_lunch_rec,total_exp,score,year
0,1,Deane-Porter Elementary School,"PK, KG-3",Rumson Borough School District,384,0.0%,"$11,020",98.3,2017
1,2,School 28,"PK, KG-8",Paterson Public School District,488,82.6%,"$9,579",97.8,2017
2,3,Thomas Edison EnergySmart Charter School,K-12,Thomas Edison EnergySmart Charter School,421,9.7%,"$7,387",96.5,2017
3,4,Terence C. Reilly School No 7,2-8,Elizabeth Public Schools,"1,055",73.5%,"$7,777",96.2,2017
4,5,Deerfield School,K-4,Millburn Township School District,563,0.5%,"$13,641",96.1,2017


In [232]:
rank_df.tail()

,rank,school,grades,district,students,free_lunch_rec,total_exp,score,year
1314,1315,Yorkship Elementary School,"PK, KG-5",Camden City School District,485,66.4%,"$16,425",2.4,2017
1315,1316,Charles Sumner Elementary School,"PK, KG-6",Camden City School District,275,62.2%,,2.3,2017
1316,1317,Camden Community Charter School,K-8,Camden Community Charter School,851,92.8%,,2.2,2017
1317,1318,Henry B. Wilson Family School,K-8,Camden City School District,555,69.2%,"$17,638",2.1,2017
1318,1319,Dr. Martin Luther King Junior Elementary School,K-5,Trenton Public School District,725,92.6%,"$9,252",1.7,2017


In [233]:
rank_df.to_csv('ele_school_summ_2017.csv', index=False)

# Demographic

In [79]:
# visit the NJ schools url
nj_school_rate_URL = 'https://www.schooldigger.com/go/NJ/schoolrank.aspx'
browser.visit(nj_school_rate_URL)
time.sleep(5)
demo_dfs=[]

In [234]:
# create HTML object
html = browser.html
# parse HTML with BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')
total_records = int(soup.find('span' ,class_='tRows').text)
total_records

1319

In [235]:
total_pages = int(np.ceil(total_records/100))
total_pages

14

In [237]:
demo_dfs=[]
# browser.find_by_xpath('//*[@id="tabSchooList_paginate"]/ul/li[1]/a').click()
for x in range(1,total_pages+1):
    # create HTML object
    html = browser.html
    # parse HTML with BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')
    # access html elements to get the link for the table data
    div = soup.find(id= 'tabSchooList_wrapper')
    table = div.find('table', id = 'tabSchooList')
    tbody = table.find('tbody')
    listings= tbody.find_all('td')
    rank = []
    school = []
    types=[]
    grades = []
    address=[]
    city=[]
    zip_cd=[]
    county=[]
    district = []
    year = []
    for i in range(0,len(listings),14):
        rank.append(listings[i].text) 
        year.append(2017)
    for i in range(1,len(listings),14):
        school.append(listings[i].text.replace('\n',''))     
    for i in range(2,len(listings),14):
        types.append(listings[i].text.replace('\n',''))
    for i in range(3,len(listings),14):
        grades.append(listings[i].text.replace('\n',''))
    for i in range(4,len(listings),14):
        address.append(listings[i].text.replace('\n',''))
    for i in range(5,len(listings),14):
        city.append(listings[i].text.replace('\n',''))
    for i in range(6,len(listings),14):
        zip_cd.append(listings[i].text.replace('\n',''))
    for i in range(7,len(listings),14):
        county.append(listings[i].text.replace('\n',''))
    for i in range(8,len(listings),14):
        district.append(listings[i].text.replace('\n',''))
    df = pd.DataFrame({'rank':rank, 'school':school, 'type': types, 'grades':grades,\
                       'address':address, 'city':city, 'zip':zip_cd, 'county':county,\
                       'district':district, 'year':year})
    demo_dfs.append(df)
    print(f'Page {x} scraped!')
    # use splinter to click on the button to go to next page
    if x == total_pages:
        break
    elif x <= 4:
        browser.find_by_xpath(f'//*[@id="tabSchooList_paginate"]/ul/li[{x+1}]/a').click()
    elif (x > 4 and x < int(np.ceil(total_records/100))-2):
        browser.find_by_xpath('//*[@id="tabSchooList_paginate"]/ul/li[5]/a').click()
    elif (x > 4 and x == int(np.ceil(total_records/100))-2) or (x==5 and total_pages <=7):
        browser.find_by_xpath('//*[@id="tabSchooList_paginate"]/ul/li[6]/a').click()
    elif (x > 4 and x == int(np.ceil(total_records/100))-1) or (x==6 and total_pages <=7):
        browser.find_by_xpath('//*[@id="tabSchooList_paginate"]/ul/li[7]/a').click()    
    else:
        continue
    time.sleep(5)
demo_df = pd.concat(demo_dfs,ignore_index = True)
demo_df.head()

Page 1 scraped!
Page 2 scraped!
Page 3 scraped!
Page 4 scraped!
Page 5 scraped!
Page 6 scraped!
Page 7 scraped!
Page 8 scraped!
Page 9 scraped!
Page 10 scraped!
Page 11 scraped!
Page 12 scraped!
Page 13 scraped!
Page 14 scraped!


,rank,school,type,grades,address,city,zip,county,district,year
0,1,Deane-Porter Elementary School,Public,"PK, KG-3",50 Blackpoint Road,Rumson,07760,Monmouth County,Rumson Borough School District,2017
1,2,School 28,Public,"PK, KG-8",200 Presidential Boulevard,Paterson,07522,Passaic County,Paterson Public School District,2017
2,3,Thomas Edison EnergySmart Charter School,"Public, Charter, Alternative",K-12,150 Pierce Street,Somerset,08873,Somerset County,Thomas Edison EnergySmart Charter School,2017
3,4,Terence C. Reilly School No 7,Public,2-8,436 First Avenue,Elizabeth,07206,Union County,Elizabeth Public Schools,2017
4,5,Deerfield School,Public,K-4,26 Troy Lane,Short Hills,07078,Essex County,Millburn Township School District,2017


In [238]:
# demo_df = pd.concat(demo_dfs,ignore_index = True)
# demo_df.head()
demo_df.tail()

,rank,school,type,grades,address,city,zip,county,district,year
1314,1315,Yorkship Elementary School,Public,"PK, KG-5",1251 Collings Road,Camden,08104,Camden County,Camden City School District,2017
1315,1316,Charles Sumner Elementary School,Public,"PK, KG-6",8th & Jackson Sts,Camden,08104,Camden County,Camden City School District,2017
1316,1317,Camden Community Charter School,"Public, Charter",K-8,9th & Linden Streets,Camden,08102,Camden County,Camden Community Charter School,2017
1317,1318,Henry B. Wilson Family School,Public,K-8,2250 South 8th Street,Camden,08104,Camden County,Camden City School District,2017
1318,1319,Dr. Martin Luther King Junior Elementary School,Public,K-5,159 N Clinton Avenue,Trenton,08609,Mercer County,Trenton Public School District,2017


In [239]:
demo_df.to_csv('ele_school_demo_2017.csv',index=False)

In [240]:
# close the browser
browser.quit()